# Getting Just the Matching Model Paths
* Creates a list of dictionaries with the matching models for each experiment.
* Data is then used in zec_02_regridding...

* 6th May
     * Updateing to only use dictionary. Previous version had a csv file with only tas. Making a new version that stores all output.

TODO:
* Currently this is only really working for piControl and 1000pgc. Need to add the 1%CO2 runs to this
* Will eventually need to create the flows for other variables. This is perhaps partially setup.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from importlib import reload
from glob import glob
import json
from pprint import pprint
sys.path.append(os.path.join(os.getcwd(), 'Documents/PhD'))
sys.path.append('modules')
import constants

from typing import List

# Paths

The code below can be used to ge the paths of all the files. However, this can be ery slow (3 hours) for piControl. 

In [3]:
# TODO: This should be a dict not a csv so you don't have to drop nans
# fpaths_df = pd.read_csv(os.path.join(os.getcwd(),  'Documents/PhD', 'data/fpaths_picontrol.csv'))#[['picontrol', '1000pgc']]
# fpaths_df.head()
# fpaths_dict = fpaths_df.to_dict(orient='list')
# fpaths_dict['1000pgc'] = [v for v in fpaths_dict['1000pgc'] if isinstance(v, str)]
# fpaths_dict['1pct_rad'] = [v for v in fpaths_dict['1pct_rad'] if isinstance(v, str)]

# fpaths_dict.keys()

# unique_picontrol_paths = np.unique(list(map(os.path.dirname, fpaths_dict['picontrol'])))
# unique_picontrol_paths[:5]

In [76]:
with open(os.path.join(os.getcwd(),  'Documents/PhD/data', 'raw_paths.json'), 'r') as file:
    fpaths_dict = json.loads(file.read())

In [77]:
variable = 'tas'

In [78]:
fpaths_dict.keys()

dict_keys(['picontrol', '1000pgc', '1pct_rad', '1pct_bgc', '1pct'])

In [79]:
pprint(list(enumerate(fpaths_dict['picontrol'][variable][:3], start=1)), depth=2)

[(1,
  '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_314801-319612.nc'),
 (2,
  '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_246901-249512.nc'),
 (3,
  '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_236901-246812.nc')]


In [80]:
print(fpaths_dict['picontrol'][variable][:3], fpaths_dict['1000pgc'][variable][:3],
      fpaths_dict['1pct_rad'][variable][:3], fpaths_dict['1pct_bgc'][variable][:3],
      sep='\n\n\n')

['/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_314801-319612.nc', '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_246901-249512.nc', '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_236901-246812.nc']


['/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815/tas_Amon_MPI-ESM1-2-LR_esm-1pct-brch-1000PgC_r1i1p1f1_gn_207501-209412.nc', '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815/tas_Amon_MPI-ESM1-2-LR_esm-1pct-brch-1000PgC_r1i1p1f1_gn_205501-207412.nc', '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815/tas_Amon_MPI-ESM1-2-LR_esm-1pct-brch-1000PgC_r1i1p1f1_gn_199501-201

In [81]:
def zecmip_open_matching_path(fpath: str, path_list: List[str], name=None) -> str:
    """
    Find a matching path from the given list based on the model and ensemble member extracted from fpath.
    Args:
        fpath (str): The path to extract model and ensemble member information from.
        path_list (List[str]): The list of potential paths to search for a match.
    Returns:
        str: The matched path if found, otherwise np.nan.
    """
    split_path = fpath.split('/')
    model = split_path[8]
    ensemble_member = split_path[10]
    print(f'Attempting to find {model=} and {ensemble_member=}', end='')

    for potential_match in path_list:
        if model in potential_match and ensemble_member in potential_match: 
            print('- Yes')
            if name: return {model: {'1000PgC': fpath, name:potential_match}}
            else: return {model: [fpath, potential_match]}
            #return potential_match
    print('- No')
    return np.nan


def find_all_matches(list_to_match: List[str], list_of_potential_matches: List[str], name=None) -> List[dict]:
    """
    Find all matches between the given list and the potential matches list using zecmip_open_matching_path function.

    Args:
        list_to_match (List[str]): The list of paths to match against the potential matches.
        list_of_potential_matches (List[str]): The list of potential matches to search within.
    Returns:
        List[dict]: A list of dictionaries containing the matched path and its corresponding potential match.
    """
    path_matches = []
    for fpath in list_to_match:
        match = zecmip_open_matching_path(fpath, list_of_potential_matches, name)
        if isinstance(match, dict):
            path_matches.append(match)#({fpath: match})
    
    return path_matches

In [82]:
# Unique abspath (no filename).
base_fpaths_1000PgC = np.sort(np.unique(list(map(os.path.dirname, fpaths_dict['1000pgc'][variable]))))
base_fpaths_1000PgC[:3]

array(['/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/day/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429'],
      dtype='<U111')

In [ ]:
single_path = 

In [87]:
base_fpaths_1000PgC

array(['/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/day/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/day/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/day/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r4i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r4i1p2f1/day/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r5i1p2f1/Amon/tas/gn/v20190429',
     

In [85]:
match_1pct = find_all_matches(base_fpaths_1000PgC, fpaths_dict['1pct_rad'][variable], name='1pct')

Attempting to find model='CanESM5' and ensemble_member='r1i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r1i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r2i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r2i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r3i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r3i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r4i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r4i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r5i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r5i1p2f1'- No
Attempting to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'- Yes
At

In [84]:
match_1pct_rad = find_all_matches(base_fpaths_1000PgC, fpaths_dict['1pct_rad'][variable], name='1pct_rad')

Attempting to find model='CanESM5' and ensemble_member='r1i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r1i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r2i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r2i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r3i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r3i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r4i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r4i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r5i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r5i1p2f1'- No
Attempting to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'- Yes
At

In [44]:
match_picontrol = find_all_matches(base_fpaths_1000PgC, fpaths_dict['picontrol'][variable], name='picontrol')

Attempting to find model='CanESM5' and ensemble_member='r1i1p2f1'- Yes
Attempting to find model='CanESM5' and ensemble_member='r1i1p2f1'- Yes
Attempting to find model='CanESM5' and ensemble_member='r2i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r2i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r3i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r3i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r4i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r4i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r5i1p2f1'- No
Attempting to find model='CanESM5' and ensemble_member='r5i1p2f1'- No
Attempting to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'- Yes
Attempting to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'- Yes


In [45]:
len(match_picontrol)

13

In [41]:
len(match_1pct)

8

In [72]:
match_picontrol[:2]

[{'CanESM5': {'1000PgC': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
   'picontrol': '/g/data/oi10/replicas/CMIP6/CMIP/CCCma/CanESM5/piControl/r1i1p2f1/Amon/tas/gn/v20190429/tas_Amon_CanESM5_piControl_r1i1p2f1_gn_620101-640012.nc'}},
 {'CanESM5': {'1000PgC': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/day/tas/gn/v20190429',
   'picontrol': '/g/data/oi10/replicas/CMIP6/CMIP/CCCma/CanESM5/piControl/r1i1p2f1/Amon/tas/gn/v20190429/tas_Amon_CanESM5_piControl_r1i1p2f1_gn_620101-640012.nc'}}]

In [86]:
[list(obj) for obj in match_1pct]

[['MIROC-ES2L'],
 ['MIROC-ES2L'],
 ['UKESM1-0-LL'],
 ['UKESM1-0-LL'],
 ['MPI-ESM1-2-LR'],
 ['NorESM2-LM'],
 ['NorESM2-LM'],
 ['GFDL-ESM4']]

In [70]:
for pctobj in match_1pct:
    # print(piobj)
    model = list(pctobj.keys())[0]
    print(model)

MIROC-ES2L
MIROC-ES2L
UKESM1-0-LL
UKESM1-0-LL
MPI-ESM1-2-LR
NorESM2-LM
NorESM2-LM
GFDL-ESM4


# OLD

In [ ]:
path_matches = []
for fpath in base_fpaths_1000PgC:
    match = zecmip_open_matching_path(fpath, fpaths_dict['picontrol'][variable], 'piControl')
    path_matches.append({fpath:match})

In [ ]:
# These are all of the base models - models may have different experminents (r1i1p2f1, r2i1p2f1)
base_group = np.unique(['/'.join(f.split('/')[:10])  for f in base_fpaths_1000PgC])
base_group

In [ ]:
# Get one experiment for each modelling group
needed_experiments = []
# Looping through all the base paths
for base_group_single in base_group:
    
    # All the experiments that have this base path
    experiments = [f for f in base_fpaths_1000PgC if base_group_single in f]
    # Get just the first one available
    needed_experiments.append(experiments[0])
    
needed_experiments

# Old

In [ ]:
# def zecmip_open_matching_path(fpath:str, path_list: List[str]) -> str:
#     split_path = fpath.split('/')
#     model = split_path[8]; ensemble_member = split_path[10]
#     print(f'Attemping to find {model=} and {ensemble_member=}', end='')
#     for potential_match in path_list:
#         if model in potential_match and ensemble_member in potential_match: 
#             print('- Yes')
#             return potential_match
#     print('- No')
#     return np.nan

# def find_all_matches(list_to_match:List[str], list_of_potentail_matches):
    
#     path_matches = []
#     for fpath in list_to_match:
#         match = zecmip_open_matching_path(list_of_potentail_matches)
#         if isinstance(match, str): path_matches.append({fpath:match})
#     return path_matches

In [15]:
# def zecmip_open_matching_path(fpath: str, 
#                             path_list: List[str],
#                             experiment:str='esm-1pct-brch-1000PgC', debug=False) -> str:
    
#     '''This function takes a path, and finds the matching path for piControl from a list of paths
#     containing piControl paths.
    
#     Parameters
#     ----------
#     fpath: the path to the dataset that you want to match to piControl.
#     unique_picontrol_paths: paths to different piControl runs.
#     experiment: the experiment of what you are matching.
    
    
    
#     Sample of split path
#     fpath = '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429'
#     fpath.split('/')
#     0-, 1-g, 2-data, 3-oi10, 4-replicas, 5-CMIP6, 6-C4MIP, 7-CCCma, 8-CanESM5, 9-esm-1pct-brch-1000PgC, 10-r1i1p2f1, 11-Amon,       12-tas, 13-gn, 14-v20190429
    
#     '''
#     import re
#     print(f'Attempting to open matching path for:\n{fpath}')
        
#     path_branch = '/'.join(fpath.split('/')[7:-2])
#     if debug: print(f'{path_branch=}')
#     path_branch = path_branch.replace(experiment, '\w+')#'piControl|esm-piControl')
#     if debug: print(f'Searching for branch containing\n{path_branch}')
        
#     PATH_FOUND = False
#     for path in path_list:
#         #if path_branch in pi_path:
#         core_path = '/'.join(path.split('/')[7:-2])
#         if debug:print(f'{path_branch=} - {core_path=}')
#         if re.search(path_branch, core_path):
#             PATH_FOUND = True
#             if debug: print(f'Found branch:\n{path_branch}')
#             break
#     if not PATH_FOUND: return np.nan
#     if debug:  print(f'Found path:\n{picontrol_path}')
#     return path

In [ ]:
needed_model_dict = {v.split('/')[8]:v for v in needed_experiments}
needed_model_dict

In [ ]:
zecmip_path_dict = {'1000pgc': {'all': unique_1000PgC_paths.tolist(), 'needed': needed_model_dict}
                    , 'picontrol': unique_picontrol_paths.tolist()}
zecmip_path_dict.keys()

In [ ]:
pprint(zecmip_path_dict)

In [ ]:
with open('data/zecmip_model_paths.json', 'w') as f:
    json.dump(zecmip_path_dict, f)